In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import json

with open('intents.json') as f:
    data = json.load(f)

df = pd.DataFrame(data['intents'])
df

,tag,patterns,responses
0,loneliness,"[I am feeling lonely, I am so lonely, I feel s...",[The Lonely City: Adventures in the Art of Bei...
1,Overwhelm,[I'm feeling overwhelmed and just need some sp...,[The Power of Now: A Guide to Spiritual Enligh...
2,General sadness,"[I feel down, I feel sad, I'm finding it hard ...",[The Noonday Demon: An Atlas of Depression by ...
3,Emotional distress,"[I'm feeling really down today., I'm feeling s...","[The Body Keeps the Score: Brain, Mind, and Bo..."
4,Work stress,[I'm feeling burnt out and exhausted from work...,"[The 5 AM Club: Own Your Morning, Elevate Your..."
5,Academic stress,"[I'm so stressed out about my exams., I have s...",[How to Read a Book by Mortimer Adler and Char...
6,Relationship stress,[I'm stressed out about my relationship with m...,[Hold Me Tight: Seven Conversations for a Life...
7,General stress,"[I'm feeling like I'm at my breaking point., I...",[Why Zebras Don't Get Ulcers: An Updated Guide...
8,worthless,"[I feel so worthless., No one likes me., I can...",[The Gifts of Imperfection: Let Go of Who You ...
9,depressed,"[I can't take it anymore, I am so depressed, I...",[Feeling Good: The New Mood Therapy by David D...


In [3]:
dic = {"tag":[], "patterns":[], "responses":[]}
for i in range(len(df)):
    ptrns = df[df.index == i]['patterns'].values[0]
    rspns = df[df.index == i]['responses'].values[0]
    tag = df[df.index == i]['tag'].values[0]
    for j in range(len(ptrns)):
        dic['tag'].append(tag)
        dic['patterns'].append(ptrns[j])
        dic['responses'].append(rspns)

df = pd.DataFrame.from_dict(dic)
df

,tag,patterns,responses
0,loneliness,I am feeling lonely,[The Lonely City: Adventures in the Art of Bei...
1,loneliness,I am so lonely,[The Lonely City: Adventures in the Art of Bei...
2,loneliness,I feel so lonely,[The Lonely City: Adventures in the Art of Bei...
3,loneliness,I don't have anyone,[The Lonely City: Adventures in the Art of Bei...
4,loneliness,"I feel like no one really understands me, and ...",[The Lonely City: Adventures in the Art of Bei...
...,...,...,...
141,suicide,I'm feeling like I'm trapped and can't escape ...,"[Reasons to Stay Alive by Matt Haig, The Happi..."
142,suicide,I'm feeling like I'm in so much emotional pain...,"[Reasons to Stay Alive by Matt Haig, The Happi..."
143,suicide,I'm feeling like I don't have anything to live...,"[Reasons to Stay Alive by Matt Haig, The Happi..."
144,suicide,I'm feeling like suicide is the only way out.,"[Reasons to Stay Alive by Matt Haig, The Happi..."


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower=True, split=' ')
tokenizer.fit_on_texts(df['patterns'])
tokenizer.get_config()

vacab_size = len(tokenizer.word_index)
print('number of unique words = ', vacab_size)

2023-03-23 17:26:41.261666: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


number of unique words =  315


In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

ptrn2seq = tokenizer.texts_to_sequences(df['patterns'])
X = pad_sequences(ptrn2seq, padding='post')
print('X shape = ', X.shape)

lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df['tag'])
print('y shape = ', y.shape)
print('num of classes = ', len(np.unique(y)))

X shape =  (146, 18)
y shape =  (146,)
num of classes =  15


In [6]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, LayerNormalization, Dense, Dropout
from tensorflow.keras.utils import plot_model

model = Sequential()
model.add(Input(shape=(X.shape[1])))
model.add(Embedding(input_dim=vacab_size+1, output_dim=100, mask_zero=True))
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(32))
model.add(LayerNormalization())
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(np.unique(y)), activation="softmax"))
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

model.summary()
plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18, 100)           31600     
                                                                 
 lstm (LSTM)                 (None, 18, 32)            17024     
                                                                 
 layer_normalization (LayerN  (None, 18, 32)           64        
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 18, 32)            8320      
                                                                 
 layer_normalization_1 (Laye  (None, 18, 32)           64        
 rNormalization)                                                 
                                                                 
 lstm_2 (LSTM)               (None, 32)                8

In [7]:
model_history = model.fit(x=X,
                          y=y,
                          batch_size=50,
                          callbacks=[tensorflow.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)],
                          epochs=30)

Epoch 1/30
3/3 [==============================] - 13s 45ms/step - loss: 3.3728 - accuracy: 0.0959
Epoch 2/30
3/3 [==============================] - 0s 50ms/step - loss: 2.8097 - accuracy: 0.1575
Epoch 3/30
3/3 [==============================] - 0s 49ms/step - loss: 2.3734 - accuracy: 0.2466
Epoch 4/30
3/3 [==============================] - 0s 50ms/step - loss: 2.0406 - accuracy: 0.3356
Epoch 5/30
3/3 [==============================] - 0s 47ms/step - loss: 1.8564 - accuracy: 0.3836
Epoch 6/30
3/3 [==============================] - 0s 40ms/step - loss: 1.6127 - accuracy: 0.4589
Epoch 7/30
3/3 [==============================] - 0s 39ms/step - loss: 1.4306 - accuracy: 0.5000
Epoch 8/30
3/3 [==============================] - 0s 49ms/step - loss: 1.1886 - accuracy: 0.6438
Epoch 9/30
3/3 [==============================] - 0s 40ms/step - loss: 1.1017 - accuracy: 0.6918
Epoch 10/30
3/3 [==============================] - 0s 42ms/step - loss: 1.0051 - accuracy: 0.7123
Epoch 11/30
3/3 [===========

In [8]:
import re
import random

def model_responce(query): 
    text = []
    txt = re.sub('[^a-zA-Z\']', ' ', query)
    txt = txt.lower()
    txt = txt.split()
    txt = " ".join(txt)
    text.append(txt)
        
    x_test = tokenizer.texts_to_sequences(text)
    x_test = np.array(x_test).squeeze()
    x_test = pad_sequences([x_test], padding='post', maxlen=X.shape[1])
    y_pred = model.predict(x_test)
    y_pred = y_pred.argmax()
    tag = lbl_enc.inverse_transform([y_pred])[0]
    responses = df[df['tag'] == tag]['responses'].values[0]
    tag = df[df['tag'] == tag]['tag'].values
    print("you: {}".format(query))
    print("model: {}".format(random.choice(responses)))


In [9]:
model_responce("I am feeling stressed");

1/1 [==============================] - 4s 4s/step
you: I am feeling stressed
model: The Year of Magical Thinking by Joan Didion


In [10]:
import pickle

In [14]:
file_name = 'solutionChalange.sav'
pickle.dump(model,open(file_name,'wb'))

In [15]:
load_model = pickle.load(open('solutionChalange.sav','rb'))

In [16]:
model_responce("I am feeling like I want to kill my Teacher");

1/1 [==============================] - 0s 23ms/step
you: I am feeling like I want to kill my Teacher
model: Reasons to Stay Alive by Matt Haig
